In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


In [4]:
arquivo_excel = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC\Scripts\Arquivos Brutos\RELATOS\Reportes.xlsx"

df = pd.read_excel(arquivo_excel)
df = df.rename(columns=lambda x: x.strip().upper())

geolocator = Nominatim(user_agent="geo_goiania")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [5]:
LAT_MIN, LAT_MAX = -16.828, -16.546
LON_MIN, LON_MAX = -49.372, -49.043

def geocode_goiania_local(row):
    
    if pd.isna(row.get('LATITUDE')) or pd.isna(row.get('LONGITUDE')):
        endereco = f"{row['LOCAL']}, Goiânia, GO, Brasil"
        location = geocode(endereco)
        
        if location:
            lat, lon = location.latitude, location.longitude
            
            if not (LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX):
                lat, lon = -16.6869, -49.2648  
        else:
            lat, lon = -16.6869, -49.2648  

        return pd.Series([lat, lon])
    else:
        return pd.Series([row['LATITUDE'], row['LONGITUDE']])


In [6]:
df[['LATITUDE', 'LONGITUDE']] = df.apply(geocode_goiania_local, axis=1)

In [7]:
df.to_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC\Scripts\Arquivos Processados\RELATOS_GEOCODED.xlsx",
            index=False)

In [ ]:
import folium

df = pd.read_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC\Scripts\Arquivos Processados\RELATOS_GEOCODED.xlsx")


mapa = folium.Map(location=[-16.6869, -49.2648], zoom_start=12)  

for _, row in df.iterrows():
    if pd.notna(row['LATITUDE']) and pd.notna(row['LONGITUDE']):
        folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            popup=f"{row['LOCAL']}"
        ).add_to(mapa)


mapa.save("mapa_relatos.html")
